In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

In [10]:
weather_city_data = pd.read_csv("output_data/cities.csv")
weather_city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.0153,27.9116,59.94,94,0,5.75,ZA,1661534417
1,Busselton,-33.6500,115.3333,54.46,85,29,10.94,AU,1661534587
2,Codrington,-38.2667,141.9667,50.04,77,100,5.01,AU,1661534589
3,Rikitea,-23.1203,-134.9692,68.50,57,88,24.11,PF,1661534590
4,Batemans Bay,-35.7167,150.1833,53.10,88,84,4.45,AU,1661534590


In [3]:
# humidity heat map

In [11]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

locations = weather_city_data[["Lat", "Lng"]]
humidity = weather_city_data["Humidity"]
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
plt.savefig("Humidity heat map.png")

<Figure size 432x288 with 0 Axes>

In [ ]:
# narrow down cities

In [12]:
ideal_cities = weather_city_data.loc[(weather_city_data["Max Temp"]<=80) & (weather_city_data["Max Temp"]>= 70) &
                                (weather_city_data["Wind Speed"]<10) & (weather_city_data["Cloudiness"]==0)].dropna()
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
93,Castro,-24.7911,-50.0119,73.80,40,0,6.67,BR,1661534638
101,Kamari,36.8000,25.8167,76.10,67,0,3.18,GR,1661534642
161,Brits,-25.6347,27.7802,72.28,21,0,3.33,ZA,1661534676
193,Taft,31.7471,54.2088,73.35,13,0,2.30,IR,1661534696
201,Severnyy,50.6770,36.5532,77.04,36,0,8.95,RU,1661534701
205,Yangi Marg`ilon,40.4272,71.7189,72.32,30,0,4.61,UZ,1661534703
249,Cap Malheureux,-19.9842,57.6142,70.47,70,0,8.99,MU,1661534730
260,Solhan,38.9634,41.0293,75.42,23,0,3.85,TR,1661534735
312,Oakdale,44.9630,-92.9649,79.65,55,0,6.91,US,1661534513
349,La Ronge,55.1001,-105.2842,77.25,44,0,5.75,CA,1661534795


In [ ]:
# hotel map

In [13]:
hotel_df = ideal_cities.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
93,Castro,BR,-24.7911,-50.0119,
101,Kamari,GR,36.8000,25.8167,
161,Brits,ZA,-25.6347,27.7802,
193,Taft,IR,31.7471,54.2088,
201,Severnyy,RU,50.6770,36.5532,
205,Yangi Marg`ilon,UZ,40.4272,71.7189,
249,Cap Malheureux,MU,-19.9842,57.6142,
260,Solhan,TR,38.9634,41.0293,
312,Oakdale,US,44.9630,-92.9649,
349,La Ronge,CA,55.1001,-105.2842,


In [14]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"The closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Not found")
        
    print("------------")

print("-------Done Searching-------")

Retrieving Results for Index 93: Castro.
The closest hotel in Castro is Central Palace Hotel.
------------
Retrieving Results for Index 101: Kamari.
The closest hotel in Kamari is Kaerati Apartments.
------------
Retrieving Results for Index 161: Brits.
The closest hotel in Brits is Copperwood Hotel and Conferencing.
------------
Retrieving Results for Index 193: Taft.
The closest hotel in Taft is Shirkooh hotel Taft.
------------
Retrieving Results for Index 201: Severnyy.
Not found
------------
Retrieving Results for Index 205: Yangi Marg`ilon.
The closest hotel in Yangi Marg`ilon is TANTANA HOTEL FERGANA.
------------
Retrieving Results for Index 249: Cap Malheureux.
The closest hotel in Cap Malheureux is LUX* Grand Baie.
------------
Retrieving Results for Index 260: Solhan.
Not found
------------
Retrieving Results for Index 312: Oakdale.
The closest hotel in Oakdale is Courtyard by Marriott St Paul Woodbury.
------------
Retrieving Results for Index 349: La Ronge.
The closest hot

In [18]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
93,Castro,BR,-24.7911,-50.0119,Central Palace Hotel
101,Kamari,GR,36.8000,25.8167,Kaerati Apartments
161,Brits,ZA,-25.6347,27.7802,Copperwood Hotel and Conferencing
193,Taft,IR,31.7471,54.2088,Shirkooh hotel Taft
201,Severnyy,RU,50.6770,36.5532,
205,Yangi Marg`ilon,UZ,40.4272,71.7189,TANTANA HOTEL FERGANA
249,Cap Malheureux,MU,-19.9842,57.6142,LUX* Grand Baie
260,Solhan,TR,38.9634,41.0293,
312,Oakdale,US,44.9630,-92.9649,Courtyard by Marriott St Paul Woodbury
349,La Ronge,CA,55.1001,-105.2842,The Harbour Inn


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))